In [3]:
!pip install -q scikit-learn

In [4]:
#Librerias
import sqlite3
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [23]:
!mkdir data
!unzip archive.zip -d data/

mkdir: no se puede crear el directorio «data»: El archivo ya existe
Archive:  archive.zip
  inflating: data/AmItheAsshole.sqlite  


In [24]:
conn = sqlite3.connect("data/AmItheAsshole.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('submission',), ('comment',)]


In [25]:
cursor.execute("PRAGMA table_info(submission);")
print("Submission table:")
print(cursor.fetchall())

cursor.execute("PRAGMA table_info(comment);")
print("Comment table:")
print(cursor.fetchall())

Submission table:
[(0, 'id', 'INTEGER', 1, None, 1), (1, 'submission_id', 'VARCHAR(255)', 1, None, 0), (2, 'title', 'TEXT', 1, None, 0), (3, 'selftext', 'TEXT', 1, None, 0), (4, 'created_utc', 'DATE', 1, None, 0), (5, 'permalink', 'TEXT', 1, None, 0), (6, 'score', 'INTEGER', 1, None, 0)]
Comment table:
[(0, 'id', 'INTEGER', 1, None, 1), (1, 'submission_id', 'INTEGER', 1, None, 0), (2, 'message', 'TEXT', 1, None, 0), (3, 'comment_id', 'VARCHAR(255)', 1, None, 0), (4, 'parent_id', 'VARCHAR(255)', 1, None, 0), (5, 'created_utc', 'DATE', 1, None, 0), (6, 'score', 'INTEGER', 1, None, 0)]


In [26]:
df_sub = pd.read_sql_query("SELECT * FROM comment LIMIT 5;", conn)
print(df_sub)

   id submission_id                                            message  \
0   1        xt1ksm  Do people with two digits to their age really ...   
1   2        xt1ksm  Lots of posts in the last 3-4 days about rando...   
2   3        xt1ksm  Sometimes I think people are making up stories...   
3   4        xt1ksm  Saw it on FB but it's hilarious how threads wi...   
4   5        xt1ksm  The OP: My MIL can be a bit petty sometimes\n\...   

  comment_id  parent_id  created_utc  score  
0    irs5v1y  t3_xt1ksm   1665421334     59  
1    isdxgsq  t3_xt1ksm   1665813660     55  
2    iryatl4  t3_xt1ksm   1665529128     40  
3    is3i5i9  t3_xt1ksm   1665622760     33  
4    ituw9ym  t3_xt1ksm   1666793880     32  


In [28]:
df_submissions = pd.read_sql_query("SELECT submission_id, title, selftext FROM submission;", conn)
df_comments = pd.read_sql_query("SELECT submission_id, message FROM comment;", conn)

In [29]:
print(len(df_comments))
print(len(df_submissions))

9101374
30994


In [30]:
import re

verdict_keywords = ['NTA', 'YTA', 'ESH', 'NAH', 'INFO']
verdict_pattern = re.compile(r'\b(' + '|'.join(verdict_keywords) + r')\b', re.IGNORECASE)

def extract_verdict(msg):
    match = verdict_pattern.search(msg)
    if match:
        return match.group(1).upper()
    return None

df_comments['label'] = df_comments['message'].apply(extract_verdict)
df_comments = df_comments[df_comments['label'].notnull()]


In [31]:
len(df_comments)

4141248

In [32]:
submission_verdicts = df_comments.groupby('submission_id')['label'].apply(lambda x: Counter(x).most_common(1)[0][0]).reset_index()

df_merged = df_submissions.merge(submission_verdicts, on='submission_id')
df_merged['text'] = df_merged['title'] + ' ' + df_merged['selftext']

In [65]:
df_merged

,submission_id,title,selftext,label,text
0,xt1ksm,AITA Monthly Open Forum Spooktober 2022,#Keep things civil. Rules still apply.\n\n##Th...,NTA,AITA Monthly Open Forum Spooktober 2022 #Keep ...
1,yiplwk,AITA for asking my friend to move a picture of...,"\n\nMe (M32) and my wife, Dahlia (F28) lost ou...",YTA,AITA for asking my friend to move a picture of...
2,yiv572,AITA for asking my husband to stay with me whi...,Throwaway my family knows my account. I'll get...,YTA,AITA for asking my husband to stay with me whi...
3,yimgaf,AITA for telling my SIL to stop talking about ...,My (37M) wife (37F) is pregnant with our first...,NTA,AITA for telling my SIL to stop talking about ...
4,yin7pf,"AITA for wanting to meet my ""daughter"" after g...",Long story short: in my (40f) twenties I had a...,YTA,"AITA for wanting to meet my ""daughter"" after g..."
...,...,...,...,...,...
30747,17vg85i,AITA for saying to my wife that I’m the one wh...,My (43M) niece (14F) is being raised by my wif...,YTA,AITA for saying to my wife that I’m the one wh...
30748,17v5nna,AITA for snapping at BIL for accusing me of st...,I (48M) was recently at my sister’s (33F) hous...,NTA,AITA for snapping at BIL for accusing me of st...
30749,17v959p,AITA for planning a spotlight dance with just ...,I (25M) will be getting married to my amazing ...,NTA,AITA for planning a spotlight dance with just ...
30750,17ventx,AITA for telling my parents that my siblings a...,"I’m catching flak from this from all sides, bu...",NTA,AITA for telling my parents that my siblings a...


In [35]:
from sklearn.model_selection import train_test_split
from collections import Counter
import torch, numpy as np, pandas as pd

# 1.1 quedarnos solo con los 5 veredictos
target_labels = ['NTA', 'YTA', 'ESH', 'NAH', 'INFO']
df = df_merged[df_merged['label'].isin(target_labels)].copy()

print(Counter(df['label']))       # inspecciona el desbalance

# 1.2 codificar etiquetas → números
label2id = {lab: i for i, lab in enumerate(target_labels)}
id2label = {i: lab for lab, i in label2id.items()}
df['label_id'] = df['label'].map(label2id)

# 1.3 dividir
train_df, tmp_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=42)
val_df, test_df  = train_test_split(tmp_df, test_size=0.5, stratify=tmp_df['label_id'], random_state=42)

print(len(train_df), len(val_df), len(test_df))


Counter({'NTA': 25027, 'YTA': 5386, 'ESH': 247, 'NAH': 133, 'INFO': 48})
24672 3084 3085


In [37]:
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig, BertModel
import torch

MODEL_NAME = 'bert-base-uncased'

tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [39]:
from torch.utils.data import Dataset, DataLoader

MAX_LEN = 256          # 256 cubre >95 % de los posts

def tokenize(batch):
    return tokenizer(batch['text'],
                     padding='max_length',
                     truncation=True,
                     max_length=MAX_LEN)

class AITADataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.encodings = tokenize(df.to_dict('list'))
        self.labels    = df['label_id'].tolist()
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_ds = AITADataset(train_df)
val_ds   = AITADataset(val_df)
test_ds  = AITADataset(test_df)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader =  DataLoader(test_ds, batch_size=32)

In [45]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, accuracy_score



def train_step(model, optimizer, total_loss, batch):
    """
    Ejecuta un solo paso de entrenamiento para un modelo y optimizador dados.
    El método realiza el pase hacia adelante del modelo con el lote de entrada, calcula la pérdida, retropropaga los gradientes y realiza un paso de optimización.
    La pérdida total se actualiza y se devuelve después de la operación.

    :param model: El modelo a entrenar.
    :param optimizer: The optimization algorithm used to update the model's parameters.
    :param total_loss: The cumulative loss that will be updated by adding the loss
        from the current batch.
    :param batch: A dictionary containing the input data for the model with keys
        'input_ids', 'attention_mask', and 'labels', required for the forward pass.
    :return: The updated total loss after adding the loss from the current batch.
    """
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    total_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return total_loss


def validation_step(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}")


def train(model, train_loader, val_loader, epochs=1):
    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.03)
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc="Training"):
            total_loss = train_step(model, optimizer, total_loss, batch)

        avg_train_loss = total_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        validation_step(model, val_loader)


    
        
model = BertForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=5
        )
model.to(device)
train(model, train_loader, val_loader, epochs=5)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/5


Training: 100%|███████████████████████████████| 771/771 [01:45<00:00,  7.28it/s]


Average training loss: 0.5430


Validation: 100%|███████████████████████████████| 97/97 [00:04<00:00, 22.26it/s]


...

Validation Accuracy: 0.7834

Epoch 5/5


Training: 100%|███████████████████████████████| 771/771 [01:48<00:00,  7.13it/s]


Average training loss: 0.2251


Validation: 100%|███████████████████████████████| 97/97 [00:04<00:00, 22.14it/s]

Validation Accuracy: 0.7740


In [51]:
def evaluate_model(model, test_loader, device, idx_to_label_map):
    model.eval()  # Poner el modelo en modo de evaluación
    all_preds = []
    all_labels = []

    with torch.no_grad():  # No necesitamos calcular gradientes durante la evaluación
        for batch in tqdm(test_loader, desc="Evaluating on Test Set"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # outputs.logits tiene la forma (batch_size, num_labels)
            # Tomamos el argmax para obtener la clase predicha
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f"\n--- Test Set Evaluation ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Weighted Precision: {precision:.4f}")
    print(f"Weighted Recall: {recall:.4f}")
    print(f"Weighted F1-Score: {f1:.4f}")

    # Reporte de clasificación más detallado
    # Mapear los índices a nombres de etiquetas para el reporte
    target_names = [idx_to_label_map[i] for i in sorted(idx_to_label_map.keys())]
    print("\nClassification Report:")
    # Es importante que all_labels y all_preds contengan los índices numéricos
    # y que target_names esté en el orden correcto de esos índices.
    # Si tus etiquetas no son 0,1,2,3,4, necesitarás ajustar 'labels' en classification_report
    unique_labels_in_data = sorted(list(set(all_labels) | set(all_preds)))
    report_target_names = [idx_to_label_map[i] for i in unique_labels_in_data]
    
    print(classification_report(all_labels, all_preds, target_names=report_target_names, labels=unique_labels_in_data, zero_division=0))
    
    return all_preds, all_labels

# --- Función para predecir en un texto nuevo ---
def predict_new_text(text_input, model, tokenizer, device, idx_to_label_map, max_len=512):
    model.eval() # Poner el modelo en modo de evaluación

    # Tokenizar el texto de entrada
    encoding = tokenizer.encode_plus(
        text_input,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length', # Asegurar padding a max_len
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',  # Retornar tensores de PyTorch
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad(): # No calcular gradientes
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    prediction_idx = torch.argmax(outputs.logits, dim=1).item() # .item() para obtener el valor Python
    predicted_label = idx_to_label_map.get(prediction_idx, "Unknown Label")

    # Opcional: obtener probabilidades
    probabilities = torch.softmax(outputs.logits, dim=1).squeeze().cpu().numpy()
    confidence = probabilities[prediction_idx]
    
    print(f"\n--- Prediction for new text ---")
    print(f"Input Text: \"{text_input[:100]}...\"") # Mostrar solo una parte si es largo
    print(f"Predicted Label: {predicted_label}")
    print(f"Confidence: {confidence:.4f}")
    # print("Probabilities per class:")
    # for i, prob in enumerate(probabilities):
    #     print(f"  {idx_to_label_map.get(i, f'Class {i}')}: {prob:.4f}")
        
    return predicted_label, confidence


# --- Ejecución ---
# Asegúrate de que 'model', 'test_loader', 'device', 'idx_to_label' (o tu mapeo)
# y 'tokenizer' estén definidos y sean los correctos.

# 1. Evaluar en el conjunto de prueba
print("\nEvaluating model on the test set...")

test_preds, test_labels = evaluate_model(model, test_loader, device, id2label)


# 2. Probar con un ejemplo nuevo
print("\nMaking a prediction on a new example...")
# Suponiendo que 'model', 'tokenizer', 'device', 'idx_to_label' están definidos.
# El tokenizer debe ser el mismo que usaste para entrenar (BertTokenizerFast.from_pretrained(MODEL_NAME))

new_post_example_1 = "AITA for telling my roommate to clean his dishes? He leaves them in the sink for days and it's starting to smell. I've asked him politely multiple times."
new_post_example_2 = "My sister wants to name her baby a name I was planning to use for my future child. I told her she couldn't, and now she's mad. AITA?"
new_post_example_3 = "I accidentally tripped someone at the grocery store. I apologized immediately and helped them up. They said it was fine. So, am I the asshole for tripping them even if it was an accident?"

# Ejemplo de cómo llamar a la función de predicción:
if 'model' in locals() and 'tokenizer' in locals() and 'device' in locals() and 'id2label' in locals():
    predicted_label_1, confidence_1 = predict_new_text(new_post_example_1, model, tokenizer, device, id2label)
    predicted_label_2, confidence_2 = predict_new_text(new_post_example_2, model, tokenizer, device, id2label)
    predicted_label_3, confidence_3 = predict_new_text(new_post_example_3, model, tokenizer, device, id2label)
else:
    print("Error: 'model', 'tokenizer', 'device', o 'idx_to_label' no están definidos para la predicción.")


Evaluating model on the test set...


Evaluating on Test Set: 100%|███████████████████| 97/97 [00:04<00:00, 22.96it/s]


--- Test Set Evaluation ---
Accuracy: 0.7750
Weighted Precision: 0.7620
Weighted Recall: 0.7750
Weighted F1-Score: 0.7685

Classification Report:
              precision    recall  f1-score   support

         NTA       0.86      0.87      0.86      2503
         YTA       0.39      0.38      0.38       539
         ESH       0.00      0.00      0.00        25
         NAH       0.00      0.00      0.00        13
        INFO       0.00      0.00      0.00         5

    accuracy                           0.78      3085
   macro avg       0.25      0.25      0.25      3085
weighted avg       0.76      0.78      0.77      3085


Making a prediction on a new example...

--- Prediction for new text ---
Input Text: "AITA for telling my roommate to clean his dishes? He leaves them in the sink for days and it's start..."
Predicted Label: NTA
Confidence: 0.5181

--- Prediction for new text ---
Input Text: "My sister wants to name her baby a name I was planning to use for my future child. I t


/home/sis.virtual.uniandes.edu.co/ca.espitiaa/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
